In [42]:
import datetime
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import StaleElementReferenceException
import subprocess
subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=0 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동

options = Options()
# 최대화면으로 설정
options.add_argument('--start-maximized')
# 꺼짐 방지
options.add_experimental_option("detach", True)
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

# 크롬 디버그 옵션 추가
options.add_argument("--remote-debugging-port=9222")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

#최대 30일까지 지원. 그 이후에는 과거 데이터로 예측, DB 구축해서 해결
now = datetime.date.today()
#현재 날짜부터 검색
now_year_date = now



#목표한 날짜. 30일정도 잡으면 될 것 같습니다
target_date = 73


df_list = []

#출발 공항 지정
arrs = ['ICN','CJJ','TAE','KWJ','PUS']
#도착 공항 지정
dets = ['NRT','HND']
lens = len('22:55–11:50')
grades = ['','premium','business','first']

for gr in grades:
    if gr == '':
        grade = 'economy'
    else:
        grade = gr
    for arr in arrs:
        for det in dets:
            for i in range(target_date):
                arr_date = now_year_date + datetime.timedelta(days=i)
                url1 = f"https://www.kayak.co.kr/flights/{arr}-{det}/{arr_date}/{gr}?sort=price_a"
                driver.get(url1)
                s = 0
                while s <=5:
                    try:
                        driver.find_element(By.CLASS_NAME,'WZTU-captcha')
                        driver.quit()
                        driver = webdriver.Chrome(service = service, options=options)
                        driver.get(url1)
                        s +=1
                    except NoSuchElementException:
                        break

                if s >= 5:
                    inp = input()

                time.sleep(random.randrange(4,15))
                #쿠키 확인 버튼이 있는지 확인
                try:
                    check_button = driver.find_element(By.CLASS_NAME, 'v-c-p.centre ')
                    driver.execute_script("arguments[0].click();", check_button)

                except NoSuchElementException:
                    pass  # 버튼이 없으므로 아무것도 하지 않고 넘어갑니다.
                try:
                    driver.find_element(By.CLASS_NAME,'ULvh-button.show-more-button').click()
                except NoSuchElementException:
                    pass

                time.sleep(random.randrange(3,8))
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                #도표를 가져옴
                data = driver.find_elements(By.CLASS_NAME,'nrc6-inner')
                s = 0

                for j in data:
                    #몇시에 출발하는지 찾습니다
                    times = j.find_element(By.CLASS_NAME,'vmXl.vmXl-mod-variant-large').text
                    #얼마인지 찾습니다
                    price = j.find_element(By.CLASS_NAME, 'f8F1-price-text').text

                    attempts = 0
                    while attempts < 5:
                        try:
                            flight_direct = j.find_element(By.CLASS_NAME,'JWEO-stops-text').text
                            break
                        except StaleElementReferenceException:
                            attempts += 1
                            continue

                    #만약 어느 하나가 비어있으면 문제가 있는 것이므로 넘어갑니다

                    if times.strip() == '' or price.strip() == '':
                        continue
                    else:
                        df_list.append([arr,det,arr_date,times[:lens],int(price.replace(",","").replace("원","")),grade,flight_direct])
                    print(arr_date,times[:lens],int(price.replace(",","").replace("원","")),grade,flight_direct)
                time.sleep(10)

2023-07-20 18:30–21:00 369896 economy 직항
2023-07-20 18:30–21:00 424400 economy 직항
2023-07-20 18:55–09:25 431600 economy 1회 경유
2023-07-20 19:05–09:55 432200 economy 1회 경유
2023-07-20 18:35–09:45 447100 economy 1회 경유
2023-07-20 18:35–11:05 447100 economy 1회 경유
2023-07-20 18:20–15:35 458718 economy 2회 경유
2023-07-20 18:20–15:35 458718 economy 2회 경유
2023-07-20 18:20–15:35 458718 economy 2회 경유
2023-07-20 22:50–06:15 962790 economy 1회 경유
2023-07-20 22:45–08:00 991600 economy 2회 경유
2023-07-20 22:45–08:00 991600 economy 2회 경유
2023-07-20 22:45–08:00 995500 economy 2회 경유
2023-07-20 22:45–08:00 1000700 economy 2회 경유
2023-07-20 22:45–08:00 1000700 economy 2회 경유
2023-07-20 22:45–08:00 1005900 economy 2회 경유
2023-07-20 22:45–08:00 1005900 economy 2회 경유
2023-07-20 22:45–08:00 1015100 economy 2회 경유
2023-07-20 19:50–15:10 1922576 economy 1회 경유
2023-07-20 19:05–12:25 3100025 economy 1회 경유
2023-07-21 17:05–19:30 177599 economy 직항
2023-07-21 09:40–12:20 177599 economy 직항
2023-07-21 07:55–10:25 189700 economy

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=114.0.5735.199); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0064A813+48355]
	(No symbol) [0x005DC4B1]
	(No symbol) [0x004E5358]
	(No symbol) [0x004E87A1]
	(No symbol) [0x004E99E1]
	(No symbol) [0x004E9A80]
	(No symbol) [0x0050D2FD]
	(No symbol) [0x0052A73C]
	(No symbol) [0x00509A36]
	(No symbol) [0x0052AA94]
	(No symbol) [0x0053C922]
	(No symbol) [0x0052A536]
	(No symbol) [0x005082DC]
	(No symbol) [0x005093DD]
	GetHandleVerifier [0x008AAABD+2539405]
	GetHandleVerifier [0x008EA78F+2800735]
	GetHandleVerifier [0x008E456C+2775612]
	GetHandleVerifier [0x006D51E0+616112]
	(No symbol) [0x005E5F8C]
	(No symbol) [0x005E2328]
	(No symbol) [0x005E240B]
	(No symbol) [0x005D4FF7]
	BaseThreadInitThunk [0x75927D59+25]
	RtlInitializeExc

In [45]:
df = pd.DataFrame(df_list, columns=['Arr','Det','Arr_date', 'times', 'price','grade','direct or nondirect'])
df.tail()

,Arr,Det,Arr_date,times,price,grade,direct or nondirect
4442,CJJ,NRT,2023-07-26,17:10–18:15,434061,economy,2회 경유
4443,CJJ,NRT,2023-07-26,17:10–18:15,434061,economy,2회 경유
4444,CJJ,NRT,2023-07-26,10:00–22:10,435145,economy,1회 경유
4445,CJJ,NRT,2023-07-26,17:10–18:15,438802,economy,2회 경유
4446,CJJ,NRT,2023-07-26,17:10–18:15,444879,economy,2회 경유


In [46]:
df.to_csv('tokyo_data.csv', index=False)